In [3]:
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import json

from netket import experimental as nkx

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [4]:
# create graph

L = 2  # take a 2x4 lattice
D = 4

t = 1  # tunneling/hopping
U = 0.01  # coulomb

# create the graph our fermions can hop on
g = nk.graph.Hypercube(length=L, n_dim=D, pbc=True)
n_sites = g.n_nodes

g

Lattice(
    n_nodes=16,
    extent=[2 2 2 2],
    basis_vectors=
        [[1. 0. 0. 0.]
         [0. 1. 0. 0.]
         [0. 0. 1. 0.]
         [0. 0. 0. 1.]],
    site_offsets=
        [[0. 0. 0. 0.]],
)

In [5]:
# create a hilbert space with 2 up and 2 down spins
hi = nkx.hilbert.SpinOrbitalFermions(n_sites, n_fermions=2)
hi

SpinOrbitalFermions(n_orbitals=16, n_fermions=2)

In [6]:
# create an operator representing fermi hubbard interactions
# -t (i^ j + h.c.) + U (i^ i j^ j)
# we will create a helper function to abbreviate the creation, destruction and number operators
# each operator has a site and spin projection (sz) in order to find the right position in the hilbert space samples
def c(site):
    return nkx.operator.fermion.destroy(hi, site)
    
def cdag(site):
    return nkx.operator.fermion.create(hi, site)

def nc(site):
    return nkx.operator.fermion.number(hi, site)

In [7]:
ham = 0.0
for u, v in g.edges():
    ham += -t * (cdag(u) * c(v) + cdag(v) * c(u))

    ham += U * nc(u) * nc(v)

In [8]:
print("Hamiltonian =", ham.operator_string())

Hamiltonian = -1.0 [3^ 7] +
-1.0 [7^ 3] +
0.01 [3^ 3 7^ 7] +
-1.0 [4^ 6] +
-1.0 [6^ 4] +
0.01 [4^ 4 6^ 6] +
-1.0 [12^ 13] +
-1.0 [13^ 12] +
0.01 [12^ 12 13^ 13] +
-1.0 [4^ 12] +
-1.0 [12^ 4] +
0.01 [4^ 4 12^ 12] +
-1.0 [5^ 7] +
-1.0 [7^ 5] +
0.01 [5^ 5 7^ 7] +
-1.0 [5^ 13] +
-1.0 [13^ 5] +
0.01 [5^ 5 13^ 13] +
-1.0 [0^ 2] +
-1.0 [2^ 0] +
0.01 [0^ 0 2^ 2] +
-1.0 [8^ 9] +
-1.0 [9^ 8] +
0.01 [8^ 8 9^ 9] +
-1.0 [8^ 12] +
-1.0 [12^ 8] +
0.01 [8^ 8 12^ 12] +
-1.0 [9^ 11] +
-1.0 [11^ 9] +
0.01 [9^ 9 11^ 11] +
-1.0 [0^ 8] +
-1.0 [8^ 0] +
0.01 [0^ 0 8^ 8] +
-1.0 [1^ 3] +
-1.0 [3^ 1] +
0.01 [1^ 1 3^ 3] +
-1.0 [1^ 9] +
-1.0 [9^ 1] +
0.01 [1^ 1 9^ 9] +
-1.0 [6^ 14] +
-1.0 [14^ 6] +
0.01 [6^ 6 14^ 14] +
-1.0 [4^ 5] +
-1.0 [5^ 4] +
0.01 [4^ 4 5^ 5] +
-1.0 [14^ 15] +
-1.0 [15^ 14] +
0.01 [14^ 14 15^ 15] +
-1.0 [0^ 1] +
-1.0 [1^ 0] +
0.01 [0^ 0 1^ 1] +
-1.0 [10^ 11] +
-1.0 [11^ 10] +
0.01 [10^ 10 11^ 11] +
-1.0 [9^ 13] +
-1.0 [13^ 9] +
0.01 [9^ 9 13^ 13] +
-1.0 [0^ 4] +
-1.0 [4^ 0] +
0.01 [0^ 0 4^ 4] 

In [9]:
occupation = np.zeros(n_sites)
occupation[0] = occupation[1] = 1
#occupation /= np.linalg.norm(occupation)
res = ham.get_conn(occupation)
res

(array([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([ 0.01,  1.  ,  1.  , -1.  , -1.  ,  1.  , -1.  ]))

In [11]:
Hdense = ham.to_dense()
i = hi.states_to_numbers(occupation)
v = np.zeros(hi.n_states)
v[i] = 1

v /= np.linalg.norm(v)

In [12]:
E = (v.conj().T).dot(Hdense.dot(v))
E

0.01